In [1]:
import json
import os
import random
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

In [2]:
with open('../videos/subreddits/AskScience/data_new.json') as file:
    data = json.load(file)

OPENAI_KEY  = os.gentenv("OPENAI_KEY")

client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key= OPENAI_KEY,
)

In [12]:
def save_script(post):
    id = post[0]
    url = post[1]
    question = post[2]
    context = post[3]
    answers = post[4]
    citation = post[5]

    long_answer = ''
    for answer in answers:
        long_answer += answer + '\n\n'

    long_citation = ''
    long_citation += url + '\n\n'
    for cite in citation:
        long_citation += cite[0] + '\n\n'
    
    script = f"--BEGIN QUESTION--\n\n{question}\\n\n--END QUESTION--\n\n" \
             f"--BEGIN CONTEXT--\n\n{context}\n\n--END CONTEXT--\n\n" \
             f"--BEGIN ANSWER--\n\n{long_answer}\n\n--END ANSWER--\n\n" \
             f"--BEGIN CITATION--\n\n{long_citation}\n\n--END CITATION--\n\n"

    with open(f'../videos/raw_scripts/{id}.txt', 'w') as file:
        file.write(script)

def make_prompt(preprompt_filename, reference_filename):
    with open(preprompt_filename, 'r') as file:
        PRE_PROMPT = file.read()
    with open(reference_filename, 'r') as file:
        post_prompt = file.read()
    return PRE_PROMPT + '\n' +  post_prompt

def chat_gpt(prompt, model_name="gpt-3.5-turbo"):
    response = client.chat.completions.create(
        model=model_name,
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content.strip()


def script_write(script_id, script_path='../videos/scripts/', model_name="gpt-3.5-turbo"):
    prompt = make_prompt('../data/gpt_prompts/script_writer.txt', f'../videos/raw_scripts/{script_id}.txt')
    script = chat_gpt(prompt, model_name)
    with open(script_path + script_id + '.txt', 'w') as file:
        file.write(script)

def get_file_names():
    folder_path = '../videos/raw_scripts'
    file_names = []
    
    for file in os.listdir(folder_path):
        if file.endswith('.txt'):
            file_name = os.path.splitext(os.path.basename(file))[0]
            file_names.append(file_name)
    
    return file_names

In [4]:
with open('../videos/subreddits/AskScience/data_new.json') as file:
    data = json.load(file)

for post in data[0:100]:
    save_script(post)

In [5]:
for name in get_file_names()[0:100]:
    file_path = '../videos/scripts/' + name + '.txt'
    if not os.path.exists(file_path):
        script_write(name)
    else:
        # print(f"File {file_path} already exists.")
        continue


In [11]:
scripts = random.sample(get_file_names(), 40)
scripts


['cakca3',
 '7vgwdg',
 'a1tkpf',
 '7hmh2c',
 '6y33j6',
 '7omaq1',
 'wjshja',
 '9wnkfv',
 '6j0tg2',
 '7jpnjl',
 '7yx524',
 '7n81em',
 '7mmz8i',
 '7trnof',
 '7v0z9e',
 'kcxjhh',
 '7sewfa',
 '908y52',
 'ch9pwm',
 '7pnuq0',
 '7xmrjn',
 'ftvhit',
 'fdcg3r',
 '74rboo',
 'd7qi0y',
 'fx3ejn',
 'ewwmem',
 'fq1rkq',
 '6o9rki',
 'mhkbe3',
 '84auzr',
 'imu81a',
 'gzb3gy',
 '6jot46',
 '6wzxql',
 'd26nwr',
 'bmzivx',
 '6efd2k',
 'm98m4q',
 '458vhd']

In [13]:
for name in scripts:
    file_path = '../videos/scripts/' + name + '.txt'
    if not os.path.exists('../videos/scriptsv2/' + name + '.txt'):
        script_write(name, script_path='../videos/scriptsv2/', model_name='gpt-4-1106-preview') 